In [2]:
from numba import *


In [19]:
class Interval(object):
    """
    A half-open interval on the real number line.
    """
    def __init__(self, lo, hi):
        self.lo = lo
        self.hi = hi

    def __repr__(self):
        return 'Interval(%f, %f)' % (self.lo, self.hi)

    @property
    def width(self):
        return self.hi - self.lo

In [20]:
from numba import types

class IntervalType(types.Type):
    def __init__(self):
        super(IntervalType, self).__init__(name='Interval')

interval_type = IntervalType()

In [21]:
from numba.extending import type_callable

@type_callable(Interval)
def type_interval(context):
    def typer(lo, hi):
        if isinstance(lo, types.Float) and isinstance(hi, types.Float):
            return interval_type
    return typer

In [22]:
from numba.extending import models, register_model

@register_model(IntervalType)
class IntervalModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ('lo', types.float64),
            ('hi', types.float64),
            ]
        models.StructModel.__init__(self, dmm, fe_type, members)

In [23]:
from numba.extending import make_attribute_wrapper

make_attribute_wrapper(IntervalType, 'lo', 'lo')
make_attribute_wrapper(IntervalType, 'hi', 'hi')

In [24]:
from numba.extending import overload_attribute

@overload_attribute(IntervalType, "width")
def get_width(interval):
    def getter(interval):
        return interval.hi - interval.lo
    return getter

In [25]:
from numba.extending import lower_builtin
from numba import cgutils

@lower_builtin(Interval, types.Float, types.Float)
def impl_interval(context, builder, sig, args):
    typ = sig.return_type
    lo, hi = args
    interval = cgutils.create_struct_proxy(typ)(context, builder)
    interval.lo = lo
    interval.hi = hi
    return interval._getvalue()

In [29]:
from numba.extending import unbox, NativeValue

@unbox(IntervalType)
def unbox_interval(typ, obj, c):
    """
    Convert a Interval object to a native interval structure.
    """
    lo_obj = c.pyapi.object_getattr_string(obj, "lo")
    hi_obj = c.pyapi.object_getattr_string(obj, "hi")
    interval = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    interval.lo = c.pyapi.float_as_double(lo_obj)
    interval.hi = c.pyapi.float_as_double(hi_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(interval._getvalue(), is_error=is_error)

In [33]:
from numba.extending import box

@box(IntervalType)
def box_interval(typ, val, c):
    """
    Convert a native interval structure to an Interval object.
    """
    interval = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    lo_obj = c.pyapi.float_from_double(interval.lo)
    hi_obj = c.pyapi.float_from_double(interval.hi)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Interval))
    res = c.pyapi.call_function_objargs(class_obj, (lo_obj, hi_obj))
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    c.pyapi.decref(class_obj)
    return res

In [34]:
from numba import jit

@jit(nopython=True)
def inside_interval(interval, x):
    return interval.lo <= x < interval.hi

@jit(nopython=True)
def interval_width(interval):
    return interval.width

@jit(nopython=True)
def sum_intervals(i, j):
    return Interval(i.lo + j.lo, i.hi + j.hi)

In [35]:
ding = Interval(1,2)

In [36]:
inside_interval(ding, 3)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1m[1] During: typing of argument at <ipython-input-34-ee6f4deb367d> (5)[0m
[1m
File "<ipython-input-34-ee6f4deb367d>", line 5:[0m
[1mdef inside_interval(interval, x):
[1m    return interval.lo <= x < interval.hi
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class '__main__.Interval'>[0m


In [15]:
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [156]:
from numba import njit
import numpy as np

@njit(parallel=True)
def testprange(x,y,ding):
    #ding_copy = np.zeros_like(ding, dtype = ding.dtype)
    for i in prange(x):
        for j in prange(y):
            ding[i][j] = i**4 + 2*j
        #print(i)
    #print(ding)
    #return ding_copy

In [157]:
x = 100000
y = 100
ding = np.zeros((int(x), int(y)), dtype=np.int64)
print(ding)
ding = testprange(x, y, ding)
print(ding)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
None


In [158]:
%%timeit

#prange parallel true
ding = np.zeros((int(x), int(y)), dtype=np.int64)
testprange(x, y,ding)

162 ms ± 38.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
%%timeit

#range parallel true
ding = np.zeros((int(x), int(y)), dtype=np.int64)

testprange(x, y,ding)

The slowest run took 11.97 times longer than the fastest. This could mean that an intermediate result is being cached.
847 ms ± 977 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
%%timeit

#range parallel false
ding = np.zeros((int(x), int(y)), dtype=np.int64)

testprange(x, y, ding)

333 ms ± 54.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
%%timeit

#prange parallel false
ding = np.zeros((int(x), int(y)), dtype=np.int64)

testprange(x,y,ding)

352 ms ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
print(ding)

[0 1 2 3 4 5 6 7 8 9]
